# Regresión lineal para predicción de progresión de diabetes
- age: Representa la edad del paciente, normalizada (no es la edad real, sino una representación estandarizada).
- sex: Sexo del paciente, representado como una variable numérica normalizada (valores entre -0.5 y 0.5).
- bmi: Índice de Masa Corporal (Body Mass Index, BMI), una medida del peso relativo al cuadrado de la altura.
- bp:
Promedio de presión arterial (Blood Pressure, BP), normalizada.
- s1: Nivel sérico de lípidos totales (colesterol total).
- s2: Nivel sérico de lipoproteínas de baja densidad (LDL, “colesterol malo”).
- s3: Nivel sérico de lipoproteínas de alta densidad (HDL, “colesterol bueno”).
- s4: Relación entre el colesterol total y HDL.
- s5: Nivel sérico de triglicéridos, representado como una medida numérica normalizada.
- s6: Nivel sérico de glucosa en ayuno, una medida clave en el monitoreo de la diabetes.

In [2]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes

# Cargar el conjunto de datos
diabetes_data = load_diabetes()

# Convertir el conjunto de datos a un DataFrame para facilitar su manipulación
df = pd.DataFrame(data=diabetes_data.data, columns=diabetes_data.feature_names)
df['target'] = diabetes_data.target  # Agregar la variable objetivo al DataFrame

# Mostrar las primeras filas del DataFrame
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [3]:
# Análisis completo del dataset 'diabetes' + Regresión lineal
# Guarda este archivo como diabetes_analysis.py o pégalo en una celda de Jupyter.

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [4]:
# 0) Directorio de salida 
out_dir = "diabetes_analysis_output"
os.makedirs(out_dir, exist_ok=True)

In [5]:
# 1) Cargar el dataset Diabetes
diabetes = load_diabetes()  # Bunch con .data, .target, .feature_names
df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
df['target'] = diabetes.target  # progresión de la enfermedad a 1 año

In [6]:
# Guardar copia completa
df.to_csv(os.path.join(out_dir, "diabetes_full.csv"), index=False)

In [7]:
# 2) Exploración inicial (EDA)
print("Primeras filas:")
print(df.head())
print("\nInformación:")
print(df.info())
print("\nDescribe:")
print(df.describe().round(4))


Primeras filas:
        age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005670 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

         s4        s5        s6  target  
0 -0.002592  0.019907 -0.017646   151.0  
1 -0.039493 -0.068332 -0.092204    75.0  
2 -0.002592  0.002861 -0.025930   141.0  
3  0.034309  0.022688 -0.009362   206.0  
4 -0.002592 -0.031988 -0.046641   135.0  

Información:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     4

In [8]:
# Matriz de correlación
corr = df.corr().round(4)
corr.to_csv(os.path.join(out_dir, "diabetes_correlation.csv"))

In [9]:
# 3) Visualizaciones simples (histogramas)
plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
plt.hist(df['bmi'], bins=20)
plt.title("bmi")
plt.subplot(1,3,2)
plt.hist(df['bp'], bins=20)
plt.title("bp")
plt.subplot(1,3,3)
plt.hist(df['s6'], bins=20)
plt.title("s6 (glucosa)")
plt.tight_layout()
plt.savefig(os.path.join(out_dir, "diabetes_histograms.png"))
plt.close()

In [10]:
# Heatmap de correlación (matplotlib puro)
plt.figure(figsize=(10,8))
plt.imshow(corr, cmap='coolwarm', interpolation='nearest', vmin=-1, vmax=1)
plt.colorbar(label='Pearson correlation')
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.index)), corr.index)
plt.title("Heatmap de correlación - Diabetes")
plt.tight_layout()
plt.savefig(os.path.join(out_dir, "diabetes_correlation_heatmap.png"))
plt.close()

In [11]:
# 4) Preparación de datos: X, y y split
X = df.drop(columns='target')
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Escalado (StandardScaler): recomendado para regresión lineal
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
# Guardar splits escalados (como CSV)
pd.DataFrame(X_train_scaled, columns=X.columns).to_csv(os.path.join(out_dir, "X_train_scaled.csv"), index=False)
pd.DataFrame(X_test_scaled, columns=X.columns).to_csv(os.path.join(out_dir, "X_test_scaled.csv"), index=False)
y_train.to_csv(os.path.join(out_dir, "y_train.csv"), index=False)
y_test.to_csv(os.path.join(out_dir, "y_test.csv"), index=False)


In [14]:
# 5) Entrenamiento de Regresión Lineal
model = LinearRegression()
model.fit(X_train_scaled, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [15]:
# Coeficientes e intercepto
coefficients = pd.DataFrame({
    "feature": X.columns,
    "coefficient": model.coef_
}).sort_values(by="coefficient", key=lambda s: s.abs(), ascending=False)
coefficients.to_csv(os.path.join(out_dir, "diabetes_coefficients.csv"), index=False)
intercept = model.intercept_

In [16]:
# 6) Evaluación en test
y_pred = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

In [17]:
metrics = pd.DataFrame({"metric": ["MAE","MSE","RMSE","R2"], "value":[mae,mse,rmse,r2]})
metrics.to_csv(os.path.join(out_dir, "diabetes_metrics.csv"), index=False)

# Guardar predicciones
pred_df = pd.DataFrame({"y_true": y_test.values, "y_pred": y_pred})
pred_df.to_csv(os.path.join(out_dir, "diabetes_predictions.csv"), index=False)

In [18]:
# 7) Gráficas de evaluación
# Predicho vs Observado
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred, alpha=0.7)
mv = min(y_test.min(), y_pred.min()); Mv = max(y_test.max(), y_pred.max())
plt.plot([mv, Mv], [mv, Mv], 'r--', label='y = x')
plt.xlabel("Observado (target)")
plt.ylabel("Predicho")
plt.title(f"Predicho vs Observado - R2={r2:.3f} RMSE={rmse:.2f}")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(out_dir, "diabetes_pred_vs_obs.png"))
plt.close()

In [19]:
# Residuales vs Predicho
residuals = y_test.values - y_pred
plt.figure(figsize=(6,4))
plt.scatter(y_pred, residuals, alpha=0.7)
plt.axhline(0, color='r', linestyle='--')
plt.xlabel("Predicho")
plt.ylabel("Residuales")
plt.title("Residuales vs Predicho")
plt.tight_layout()
plt.savefig(os.path.join(out_dir, "diabetes_residuals_vs_pred.png"))
plt.close()

In [20]:
# Barras de coeficientes
plt.figure(figsize=(8,4))
coef_plot = coefficients.set_index('feature').loc[X.columns]
plt.bar(coef_plot.index, coef_plot['coefficient'])
plt.xticks(rotation=45)
plt.ylabel("Coeficiente")
plt.title("Coeficientes del modelo")
plt.tight_layout()
plt.savefig(os.path.join(out_dir, "diabetes_coefficients_bar.png"))
plt.close()

In [21]:
# 8) Resumen textual
with open(os.path.join(out_dir, "diabetes_conclusions.txt"), "w") as f:
    f.write("Resumen del modelo de regresión lineal sobre dataset Diabetes\n\n")
    f.write(f"Intercepto: {intercept:.4f}\n")
    f.write(f"MAE={mae:.3f}, RMSE={rmse:.3f}, R2={r2:.3f}\n\n")
    f.write("Top coeficientes (abs):\n")
    f.write(coefficients.head(10).to_string(index=False))

print("Análisis completado. Archivos guardados en:", os.path.abspath(out_dir))

Análisis completado. Archivos guardados en: c:\Users\benki\Documents\Code\Aprendizake_Maquina\GitHub\regresion_lineal\diabetes_analysis_output


Interpretación rápida (qué mirar y cómo interpretarlo)

R2: proporción de varianza explicada por el modelo (valores más cercanos a 1 mejor; en datasets médicos suele ser moderado).

MAE / RMSE: error promedio en las mismas unidades del target (la unidad aquí es la medida de progresión usada por el dataset). RMSE penaliza más errores grandes.

Coeficientes: como las variables fueron estandarizadas, un coeficiente = cambio en la predicción (target) por 1 desviación estándar de la variable.

Coeficiente positivo → un aumento en esa variable (en unidades estandarizadas) aumenta la progresión esperada.

Coeficiente negativo → aumento en la variable reduce la progresión esperada.

Residuales: revisa si están centrados alrededor de 0 y sin patrón; patrones pueden indicar relaciones no lineales o heterocedasticidad.